# Diagrama de Moody interactivo (IIQ2013)

Notebook didáctico con ecuaciones, explicación y ejemplo práctico.


## 1) Ecuaciones base

Número de Reynolds:
$$Re = \frac{V D}{\nu}$$

Factor de fricción (laminar):
$$f = \frac{64}{Re}$$

Colebrook-White (turbulento):
$$\frac{1}{\sqrt{f}}=-2\log_{10}\left(\frac{\varepsilon/D}{3.7}+\frac{2.51}{Re\sqrt{f}}\right)$$

Pérdida de carga:
$$h_f=f\frac{L}{D}\frac{V^2}{2g}$$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatLogSlider

def f_swamee_jain(Re, rr):
    return 0.25/(np.log10(rr/3.7 + 5.74/(Re**0.9))**2)

def f_colebrook(Re, rr, it=30):
    Re=np.asarray(Re,dtype=float)
    f=np.maximum(f_swamee_jain(Re,max(rr,1e-12)),1e-4)
    for _ in range(it):
        inv=-2*np.log10(rr/3.7 + 2.51/(Re*np.sqrt(f)))
        f=1/(inv**2)
    return f

def f_darcy(Re, rr):
    Re=np.asarray(Re,dtype=float)
    f=np.empty_like(Re)
    lam=Re<2300
    f[lam]=64/Re[lam]
    f[~lam]=f_colebrook(Re[~lam],rr)
    return f


In [ ]:
def moody_base(ax):
    Re=np.logspace(3,8,500)
    rr_list=[0.0,1e-6,2e-6,5e-6,1e-5,2e-5,5e-5,1e-4,2e-4,5e-4,1e-3,2e-3,5e-3,1e-2,2e-2,3e-2,5e-2]
    for rr in rr_list:
        ax.plot(Re,f_darcy(Re,max(rr,1e-12)),lw=1.25,color='steelblue')

    Re_l=np.logspace(2.3,np.log10(2300),200)
    ax.plot(Re_l,64/Re_l,color='black',lw=2.2,label='Laminar: f=64/Re')

    ax.set_xscale('log'); ax.set_yscale('log')
    ax.set_xlim(1e3,1e8); ax.set_ylim(0.005,0.1)
    ax.grid(True,which='both',ls='--',alpha=0.3)
    ax.set_title('Diagrama de Moody',fontsize=18,fontweight='bold')
    ax.set_xlabel('Número de Reynolds, Re',fontsize=14)
    ax.set_ylabel('Factor de fricción Darcy, f',fontsize=14)
    ax.tick_params(axis='both', which='major', labelsize=12)

    # Eje izquierdo: 10 ticks entre 0.005 y 0.1 (sin notación científica)
    left_ticks=np.linspace(0.005,0.1,10)
    ax.set_yticks(left_ticks)
    ax.set_yticklabels([f'{t:.4f}' for t in left_ticks], fontsize=12)

    # Eje derecho: un tick por cada línea de rugosidad (sin notación científica)
    ax2=ax.twinx()
    rr_lines=np.array(rr_list[1:])
    f_ticks=1/(-2*np.log10(rr_lines/3.7))**2
    m=(f_ticks>=ax.get_ylim()[0])&(f_ticks<=ax.get_ylim()[1])
    ax2.set_yscale('log'); ax2.set_ylim(ax.get_ylim())
    ax2.set_yticks(f_ticks[m])
    ax2.set_yticklabels([f'{r:.6f}'.rstrip('0').rstrip('.') for r in rr_lines[m]], fontsize=12)
    ax2.set_ylabel('Rugosidad relativa, ε/D',fontsize=14)

def diagrama_interactivo(Re=1e5, rr=1e-3):
    fig,ax=plt.subplots(figsize=(11,7))
    moody_base(ax)
    f=float(f_darcy(np.array([Re]),rr)[0])
    ax.scatter([Re],[f],color='#444444',s=90,zorder=6)
    ax.hlines(f, Re, ax.get_xlim()[1], colors='#666666', linestyles='-', lw=1.2, zorder=4)
    ax.vlines(Re, ax.get_ylim()[0], f, colors='#666666', linestyles='-', lw=1.2, zorder=4)
    ax.text(0.98,0.97,f'f = {f:.4f}',transform=ax.transAxes,ha='right',va='top',fontsize=14,color='#222222',fontweight='bold')
    ax.text(Re*1.08,f*1.02,f'Re={Re:.2e}\nε/D={rr:.4f}',color='#333333',fontsize=12)
    plt.show()
    plt.close(fig)

interact(
    diagrama_interactivo,
    Re=FloatLogSlider(value=1e5, base=10, min=3, max=8, step=0.01, description='Re', continuous_update=False),
    rr=FloatLogSlider(value=1e-3, base=10, min=-6, max=-1.3, step=0.01, description='ε/D', continuous_update=False)
);


## 2) Ejemplo práctico

Datos: $Q=0.020$ m³/s, $D=0.10$ m, $L=120$ m, $\nu=1.0\times10^{-6}$ m²/s, $\varepsilon=0.045$ mm.


In [ ]:
Q=0.020; D=0.10; L=120.0; nu=1.0e-6; eps=0.045e-3; g=9.81
A=np.pi*D**2/4
V=Q/A
Re=V*D/nu
rr=eps/D
f=float(f_darcy(np.array([Re]),rr)[0])
hf=f*(L/D)*(V**2/(2*g))
print(f'Velocidad V = {V:.3f} m/s')
print(f'Re = {Re:.3e}')
print(f'ε/D = {rr:.3e}')
print(f'f = {f:.4f}')
print(f'hf = {hf:.3f} m')
